### **STEP-1**. AI Solution 및 Instance 등록을 위한 준비 작업
&#x1F600; **등록 할 AI Contents 의 experimental_plan.yaml 를 alo/config/ 에 준비해 둡니다.**

&#x1F600; **가상 환경을 만들어 두고, ipykernel 을 제작해 둡니다.**     

1. ALO 의 main.py 파일이 존재하는 위치에서 아래 명령어들을 순차 실행합니다.
> conda create -n {ENV-NAME} python=3.10 \
> conda init bash \
> conda activate {ENV-NAME} \
> python main.py \
> pip install ipykernel \
> pip install requests \
> python -m ipykernel install --user --name {ENV-NAME} --display-name {IPYKERNEL-NAME}

2. 본 jupyter notebook 에서, 위에서 생성한 ipykernel 을 선택 합니다. \
   가령 tcr이라는 이름의 가상환경을 만들었다면, 아래와 같이 선택합니다.

<div style="margin: 40px">
<img src="./image/ipykernel.png" width="400">
</div>


&#x1F600; **아래 STEP들을 하나씩 실행시키면서, <u>< 사용자 입력 ></u>이라고 주석 표기된 내용을 적절히 변경해주세요.**     

----

### **STEP-2**. AI Solution 이름 선택     

#### **STEP-2-1**. AI Conductor 시스템 URI 셋팅
&#x1F600; 로그인 요청 및 시스템 담당으로부터 사용 가능한 시스템 URI를 확인합니다. 
- 고객지수플랫폼 Development 
> URI: "https://aic-kic.aidxlge.com/"
- 담당서버 테스트 환경       
> URI = "http://10.158.2.243:9999/"

> 시스템 URI 및 로그인 정보, ECR TAG, ICON FILE 명 등 사용자 입력부

In [ ]:
#----------------------------------------#
#              사용자 입력                #
#----------------------------------------#
user_input ={
    # 시스템 URI
    'URI': "http://10.158.2.243:9999/", 
    
    # workspace 이름 
    'WORKSPACE_NAME': "magna-ws", # "cism-ws"
    
    # 로그인 정보 
    'LOGIN_ID': 'magna-dev', # "cism-dev"
    'LOGIN_PW': 'magna-dev@com', # "cism-dev@com"
    
    # Scope: public / private  
    'ONLY_PUBLIC': 0, # 0: public으로 등록된 solution / 1: magna-ws으로 등록된 solution
    'URI_SCOPE': 'public', #'private' #'public'
    
    # ECR에 올라갈 컨테이너 URI TAG 
    'ECR_TAG': 'latest', 
    
    # scripts/creating_ai_solution/image/ 밑에 UI에 표시될 아이콘 이미지 파일 (ex. icon.png) 를 업로드 해주세요. 
    # 이후 해당 아이콘 파일 명을 아래에 기록 해주세요.
    'ICON_FILE': 'icon.png'
}
#----------------------------------------#

> 시스템 로그인

In [ ]:
# 시각화 용 모듈 install 
!pip install tabulate

In [ ]:
# Generate RegisterUtils instance & login
from register_utils import RegisterUtils
registerer = RegisterUtils(user_input)

# AI Conductor Login
registerer.login()

> AI Solution 이름 설정 (사용자 입력)

In [ ]:
# AI Solution 이름 설정 (기존에 존재하는 중복 이름 허용X)
from IPython.display import display, HTML

# soltuion 이름 입력 받기 
display(HTML('<p style="font-size: 20px;"> < Create the name of AI Solution! > </p>'))
user_solution_name = input("- Enter the name you want to create: ")

# 기존에 존재하는 solution name과 겹치지 않는지 체크 
registerer.check_solution_name(user_solution_name)


#### **STEE-2-2**. AI Solution Name 을 AI Conductor 에 등록합니다. 
&#x1F600; 이름이 등록되면 본 jupyter 노트북 과정이 끝날 때까지 변경이 어려 울 수 있습니다. \
변경이 필요할 경우 <b> STEP-2-1 </b> 부터 다시 실행하여 주시기 바랍니다. 
 

> 기존에 workspace에 존재하는 solution 이름을 조회하여, 사용자가 입력하는 이름이 유효한지 (고유한지) 확인합니다.

In [ ]:
# 학습 용 solution_metadata.yaml 셋팅 
###################
pipeline = 'train'
###################
# check workspace (ECR, S3정보 셋팅까지 진행)
registerer.check_workspace() 
# 받아온 workspace 정보 기반으로 solution_metadata.yaml 셋팅 
registerer.set_yaml(pipeline = pipeline)

----

#### **STEP-3**. Train 용 Sample Data 등록

In [ ]:
# s3 접근확인
registerer.s3_access_check()

# s3 데이터 업로드
# 이전에 이미 해당 s3 경로에 존재하던 데이터는 지워집니다.
registerer.s3_upload()

In [ ]:
# UI에 표시될 아이콘 이미지를 s3에 업로드합니다. 
registerer.s3_upload_icon()

----

#### **STEP-4**. Train Docker Container 제작

> ALO 작업 폴더를 현재 노트북 경로로 가져옵니다. 

In [ ]:
# ALO 작업 폴더 가져오기 
registerer.set_alo()

> DOCKERFILE을 셋팅합니다. 

In [ ]:
# DOCKERFILE setting
registerer.set_docker_contatiner()

> AWS ECR에 docker 등록을 위한 repository를 생성합니다. 

In [ ]:
## ECR 등록
## docker login 실행 
registerer.set_aws_ecr(docker=True)

## buildah login 실행 (docker in docker) - sudo 권한 필요 
# tags = [
#     "Key=Company,Value=LGE",
#     "Key=Owner,Value=IC360",
#     "Key=HQ,Value=CDO",
#     "Key=Division,Value=CDO",
#     "Key=Infra Region,Value=KIC",
#     "Key=Service Mode,Value=DE",
#     "Key=Cost Type,Value=COMPUTING",
#     "Key=Project,Value=CIS",
#     "Key=Sub Project,Value=CISM",
#     "Key=System,Value=AIDX"
# ]
# sm.set_aws_ecr(docker=False, tags=tags) 

> Docker Build

In [ ]:
# docker build 
registerer.build_docker()

> ECR에 docker image를 push 합니다. 

In [ ]:
registerer.docker_push()

> solution_metadata.yaml에 container uri를 넣어줍니다.

In [ ]:
registerer.set_container_uri() 

----

#### **STEP-5**. Training 에 사용될 User Parameters 설정

> solution_metadata.yaml에 사용자 파라미터 및 artifacts 저장 경로를 넣어줍니다. 

In [ ]:
## user parameter 입력
registerer.set_candidate_parameters() # todo: selected params?
## artifact 저장 경로 지정
registerer.set_artifacts_uri()

----

#### **STEP-6**. Training 에 사용될 Cloud resource 선택

> 컴퓨팅 자원을 선택합니다. (default: standard)

In [ ]:
#----------------------------------------#
#              사용자 입력                #
#----------------------------------------#

train_resource = 'standard'

#----------------------------------------#

# 클라우드 컴퓨팅 리소스 선택 
registerer.set_resource(train_resource)

----

#### **STEP-7**. Inference 용 Sample Data 등록

In [ ]:
###################
pipeline = "inference"
###################
# inference pipeline을 solution_metadata.yaml에 추가하고, 현재 registerer의 pipeline type을 inference로 변경 
registerer.append_pipeline(pipeline)
# s3 데이터 업로드
# 이전에 있던 데이터는 지워집니다
registerer.s3_access_check()
registerer.s3_upload()

----

#### **STEP-8**. Inference 용 Docker Container 제작

> DOCKERFILE을 셋팅합니다.

In [ ]:
# DOCKERFILE setting
registerer.set_docker_contatiner()

> AWS ECR에 docker 등록을 위한 repository를 생성합니다. 

In [ ]:
## ECR 등록
## docker login 실행 
registerer.set_aws_ecr(docker=True)

## buildah login 실행 (docker in docker) 
# tags = [
#     "Key=Company,Value=LGE",
#     "Key=Owner,Value=IC360",
#     "Key=HQ,Value=CDO",
#     "Key=Division,Value=CDO",
#     "Key=Infra Region,Value=KIC",
#     "Key=Service Mode,Value=DE",
#     "Key=Cost Type,Value=COMPUTING",
#     "Key=Project,Value=CIS",
#     "Key=Sub Project,Value=CISM",
#     "Key=System,Value=AIDX"
# ]
# sm.set_aws_ecr(docker=False, tags=tags) 

> Build docker

In [ ]:
# docker build 
registerer.build_docker()


> AWS ECR에 docker 이미지를 push 합니다. 

In [ ]:
registerer.docker_push()

> solution_metadata.yaml에 container uri를 넣어줍니다.

In [ ]:
registerer.set_container_uri() # uri도 그냥 입력되게 수정

----

#### **STEP-9**. Inference 용 User Parameters 제작

> solution_metadata.yaml에 사용자 파라미터, artifacts 경로 등을 넣어줍니다.

In [ ]:
registerer.set_candidate_parameters()
registerer.set_artifacts_uri()

> solution_metadata.yaml에 model 경로를 넣어줍니다. <u>(추론 시에만 필요합니다.)</u>

In [ ]:
registerer.set_model_uri() # 주의: model은 train artifacts 경로에 존재

----

#### **STEP-10**. Inference 용 Resource 선택

> 컴퓨팅 자원을 선택합니다.

In [ ]:
#----------------------------------------#
#              사용자 입력                #
#----------------------------------------#

inference_resource = 'standard'

#----------------------------------------#

registerer.set_resource(inference_resource)

----

#### **STEP-11**. Description 추가

> solution_metadata.yaml에 description을 추가합니다.

In [ ]:
# [임시] 수정필요 
#----------------------------------------#
#              사용자 입력                #
#----------------------------------------#
user_description ={
    'overview': "AI Advisor Test",

    'input_data': "Test input s3 bucket",
    
    'output_data': "Test output s3 bucket",

    'user_parameters': "Test params",
    
    'algorithm': "ALO"
}
#----------------------------------------#

registerer.set_description(user_description)

----

#### **STEP-12**. AI Solution & Instance 등록 !!

> AI Solution 등록을 진행합니다.

In [ ]:
# wrangler 정보 등록 
# [임시] 변경 예정
registerer.set_wrangler()

In [ ]:
#----------------------------------------#
#              사용자 입력                #
#----------------------------------------#

# Edge관련 정보 등록
edgeconductor_interface = {
            'support_labeling': False,
            
            'inference_result_datatype': 'table', # 'image'
            
            'train_datatype': 'table' # 'image'
        }

#----------------------------------------#

registerer.set_edge(edgeconductor_interface)

In [ ]:
registerer.register_solution()

> Solution instance 등록

In [ ]:
registerer.register_solution_instance()

----

> <u>[임시]</u> 추후 Edge Conductor UI에서 진행 예정 

> Stream 등록 

In [ ]:
registerer.register_stream()

> stream run 요청 (train, inference)

In [ ]:
registerer.request_run_stream()

> pipeline status 조회 

In [ ]:
registerer.get_stream_status()

> 학습 완료된 artifacts download 

In [ ]:
registerer.download_artifacts()

> download 받은 학습 artifacts로 로컬 환경에서 추론해보기

In [ ]:
# 가상환경이 잘 connected 돼있는지 확인 
!which python 

In [ ]:
# 현재 작업경로 확인 
!pwd

In [ ]:
# s3로부터 다운로드받은 train artifacts를 scripts 폴더 상위 경로의 main.py랑 같은 위치로 옮기고 추론 실행 
import os
os.makedirs("./.train_artifacts",  exist_ok=True)

!tar -xvf ./train_artifacts.tar.gz -C ./.train_artifacts/
!cp -r .train_artifacts ../../
!rm -rf ./.train_artifacts

!python ../../main.py --mode inference